In [1]:
import sys
sys.path.append('../scripts/')
from Model import *

In [2]:
from multiprocessing import Pool, cpu_count

In [57]:
import numpy as np

In [3]:
variables

['pr', 'tas', 'tasmax', 'tasmin']

In [4]:
def process(model, **kwargs):
    return(model.__getattribute__('process')(**kwargs))

In [5]:
models[1]

'BCC-CSM1-1'

In [6]:
model = models[3]
scen = 'historical'

In [25]:
%%time
m = NEXModel.from_details(model=model, variable='pr', scen=scen)
m.outdir = '/media/abhi/My_Passport/NEX-GDDP-NASA-PROCESSED'
kw = {'metric': 'monmean', 'outdir': 'auto', 'outname': 'auto'}
process(m, **kw)

CPU times: user 32 ms, sys: 32 ms, total: 64 ms
Wall time: 49.2 s


In [8]:
res = []

In [9]:
pool = Pool()

In [11]:
%%time
for variable in ['pr', 'tasmax', 'tasmin']:
    m = NEXModel.from_details(model=models[4], variable=variable, scen=scen)
    m.outdir = '/media/abhi/My_Passport/NEX-GDDP-NASA-PROCESSED'
    kw = {'metric': 'monmean', 'outdir': 'auto', 'outname': 'auto'}
    res.append(pool.apply_async(process, [m], kw))
    
pool.close()
pool.join()

CPU times: user 140 ms, sys: 48 ms, total: 188 ms
Wall time: 2min 44s


In [3]:
import xarray as xr

In [4]:
sys.path.append("/home/abhi/Documents/mygit/postBC_diagnostic/src/scripts")
from recipes import *

In [13]:
ds = xr.open_mfdataset('/media/abhi/My_Passport/NEX-GDDP-NASA/ACCESS1-0/pr/*historical*')

In [18]:
res_ds = []

In [19]:
%%time
res_ds.append(monmean(ds).compute())

CPU times: user 16.2 s, sys: 1.34 s, total: 17.5 s
Wall time: 23.4 s


In [23]:
res_ds[0].resample(time='Y').mean()

<xarray.Dataset>
Dimensions:  (lat: 160, lon: 200, time: 56)
Coordinates:
  * time     (time) datetime64[ns] 1950-12-31 1951-12-31 ... 2005-12-31
  * lat      (lat) float32 0.125 0.375 0.625 0.875 ... 39.375 39.625 39.875
  * lon      (lon) float32 60.125 60.375 60.625 ... 109.375 109.625 109.875
Data variables:
    pr       (time, lat, lon) float32 3.3788423e-05 ... 7.7612085e-06

In [5]:
from helper_funcs import get_model_files

In [13]:
def process_monmean(model, variable, scen, compute=False):
    files = get_model_files(model, variable, scen)
    res = monmean(xr.open_mfdataset(files)).rename({variable:'value'})
    if compute:
        res = res.compute()
        
    return(res)

In [7]:
compute_monmean('ACCESS1-0', 'pr', 'historical')

<xarray.Dataset>
Dimensions:  (lat: 160, lon: 200, time: 672)
Coordinates:
  * time     (time) datetime64[ns] 1950-01-31 1950-02-28 ... 2005-12-31
  * lat      (lat) float32 0.125 0.375 0.625 0.875 ... 39.375 39.625 39.875
  * lon      (lon) float32 60.125 60.375 60.625 ... 109.375 109.625 109.875
Data variables:
    value    (time, lat, lon) float32 dask.array<shape=(672, 160, 200), chunksize=(1, 160, 200)>

In [9]:
variables = ['pr', 'tasmax', 'tasmin']
scens = ['historical', 'rcp45', 'rcp85']

In [10]:
modelIndex = pd.Index(models, name='model')
variableIndex = pd.Index(variables, name='variable')
scenIndex = pd.Index(scens, name='scen')

In [58]:
res = []

In [60]:
%%time
for model in models:
    for variable in variables:
        for scen in scens:
            try:
                res_monmean = process_monmean(model, variable, scen, True)
            except Exception:
                res_monmean = np.nan
                
            res.append([model, 
                        variable, 
                        scen, 
                        res_monmean]
                      )

CPU times: user 1h 22min 24s, sys: 8min 30s, total: 1h 30min 55s
Wall time: 2h 23min 40s


In [63]:
import pandas as pd

In [68]:
df = pd.DataFrame(res, columns=['model', 'variable', 'scen', 'ds'])

In [72]:
df.to_pickle('../../pickles/ALL21_monmean.pkl')

MemoryError: 

In [73]:
import _pickle as cpickle

In [ ]:
with open('../../pickles/ALL21_monmean.pkl', 'wb') as f:
    cpickle.dump(df, f)

In [45]:

df = pd.DataFrame(res)

In [71]:
df

,model,variable,scen,ds
0,ACCESS1-0,pr,historical,[value]
1,ACCESS1-0,pr,rcp45,[value]
2,ACCESS1-0,pr,rcp85,[value]
3,ACCESS1-0,tasmax,historical,[value]
4,ACCESS1-0,tasmax,rcp45,[value]
5,ACCESS1-0,tasmax,rcp85,[value]
6,ACCESS1-0,tasmin,historical,[value]
7,ACCESS1-0,tasmin,rcp45,[value]
8,ACCESS1-0,tasmin,rcp85,[value]
9,BCC-CSM1-1,pr,historical,[value]


In [50]:
res_monmeans = list(df[3])

In [52]:
%%time
for ds in res_monmeans:
    ds.compute()

CPU times: user 1min, sys: 6.2 s, total: 1min 6s
Wall time: 1min 5s


In [47]:
%%time
df[3].apply(lambda ds: ds.compute())

CPU times: user 1min, sys: 6.37 s, total: 1min 7s
Wall time: 1min 16s


0    [value]
1    [value]
2    [value]
Name: 3, dtype: object

In [11]:
import pandas as pd
import itertools

In [14]:
a = [models, variables, scens]

In [21]:
df = pd.DataFrame(list(itertools.product(*a)), columns=['model', 'variable', 'scen'])

In [29]:
%%time
process_monmean(models[0], variables[0], scens[0], True)

CPU times: user 18.3 s, sys: 1.57 s, total: 19.9 s
Wall time: 15.7 s


<xarray.Dataset>
Dimensions:  (lat: 160, lon: 200, time: 672)
Coordinates:
  * time     (time) datetime64[ns] 1950-01-31 1950-02-28 ... 2005-12-31
  * lat      (lat) float32 0.125 0.375 0.625 0.875 ... 39.375 39.625 39.875
  * lon      (lon) float32 60.125 60.375 60.625 ... 109.375 109.625 109.875
Data variables:
    value    (time, lat, lon) float32 3.6919027e-05 3.577503e-05 ... 0.0 0.0

In [26]:
%%time
df['monmean'] = df.apply(lambda r: process_monmean(r[0], r[1], r[2]), axis=1)

KeyboardInterrupt: 

In [55]:
m = NEXModel.from_details(models[0], variables[0], scens[0])

In [56]:
%%time
for scen in scens:
    NEXModel.from_details(models[0], variables[0], scen).process(metric='monmean')

CPU times: user 352 ms, sys: 416 ms, total: 768 ms
Wall time: 3min 27s
